# Tarea 3
Gabriel De La Parra

## Modulo 1 - Segmentación del Epitelio
Para implementar esta segmentación utilice una filtro de difusión anisotrópica para mejorar la continuidad de las membranas (separación entre celulas). Puede implementarlo directamente o utilizar funciones existentes. A continuación realice una segmentación automática con el método de Otsu. Utilice todos los filtros morfológicos que estime necesario para
corregir la segmentación.

En esta demostración, se entregarán primero los resultados y posteriormente el proceso paso a paso.

### Inicialización de Packages

In [ ]:
%matplotlib inline

from scipy import ndimage as ndi
from skimage import io, morphology, filters, exposure, measure, segmentation
import matplotlib.pylab as plt
import matplotlib
import numpy as np
from PM2 import anisodiff2
from ipywidgets import interact, widgets
from IPython import display

#Ajustar el tamaño de las imágenes:
matplotlib.rcParams['figure.figsize'] = (14,12)

### Interactivo - Segmentación de células

In [ ]:
images = io.imread("./images/raw.tif")
image = images[0]

def on_button_clicked(b):
    plot()
    
def plot(equalize=False, adaptEq=True, anis=True, niter=3, kappa=30, thres=True, otsu=0, morph=True, dilat=0, skel=True, remove=True, min_size=4, clear=True, Filter=True, labels=True):
    image = images[0]

    if Filter:
        image = np.invert(image)

        if equalize:
            image = exposure.equalize_hist(image)
        if adaptEq:
            image = exposure.equalize_adapthist(image)

        if anis:
            image = anisodiff2(image, niter, kappa)

        if thres:
            image = image < (filters.threshold_otsu(image) + otsu)
        
        if morph:
            for i in range(0,dilat):
                image = morphology.binary_dilation(image)
        
        if remove:
            image = morphology.remove_small_objects(image, min_size=min_size, connectivity=8)
        
        if skel:
            image = morphology.skeletonize(image)
            image = morphology.binary_dilation(image)

        if clear:
            image = np.invert(image)
            image = segmentation.clear_border(image)
    
    if Filter and labels:
        label,_ = ndi.label(image)
        regions = measure.regionprops(label)
        plt.imshow(label, cmap="gist_earth")
        print("Found:",len(regions),"regions!")
        plt.show()
    
    
    if not labels or not Filter:
        print("Image:")
        plt.imshow(image, cmap="gray")
        plt.show()
    

    
minSizeSlider = widgets.IntSlider(value=8, min=0, max=200, description='Min Size:')
otsuSlider = widgets.IntSlider(value=0, min=-20, max=20, description='Otsu Bias:')
nIterSlider = widgets.IntSlider(value=2, min=1, max=200, description='Anif N:')
kappaSlider = widgets.IntSlider(value=50, min=1, max=200, description='Anif K:')
dilatSlider = widgets.IntSlider(value=4, min=0, max=6, description='Dilat:')

interact(plot, min_size=minSizeSlider, otsu=otsuSlider, niter=nIterSlider, kappa=kappaSlider, dilat = dilatSlider)

print("Interactive filtering")

### Proceso paso a paso

#### Visualización imágen

In [ ]:
images = io.imread("./images/raw.tif")
image = images[0]
plt.imshow(image, cmap="gray")
plt.show()

#### Invertir imagen

In [ ]:
image = np.invert(image)
plt.imshow(image, cmap="gray")
plt.show()

#### Ecualización adaptativa de histograma

In [ ]:
image = exposure.equalize_adapthist(image)
plt.imshow(image, cmap="gray")
plt.show()

#### Filtro por difusión anisotrópica

In [ ]:
image = anisodiff2(image, 3, 60)
plt.imshow(image, cmap="gray")
plt.show()

#### Binarización por Otsu

In [ ]:
image = image < (filters.threshold_otsu(image))
plt.imshow(image, cmap="gray")
plt.show()

#### Filtrar pequeños objetos

In [ ]:
image = morphology.remove_small_objects(image, min_size=8, connectivity=8)
plt.imshow(image, cmap="gray")
plt.show()

#### Morfología: Dilatación para completar bordes faltantes

In [ ]:
image = morphology.binary_dilation(image)
image = morphology.binary_dilation(image)
image = morphology.binary_dilation(image)
image = morphology.binary_dilation(image)
plt.imshow(image, cmap="gray")
plt.show()

#### Esqueletización

In [ ]:
image = morphology.skeletonize(image)
plt.imshow(image, cmap="gray")
plt.show()

In [ ]:
image = morphology.binary_dilation(image)
plt.imshow(image, cmap="gray")
plt.show()

#### Eliminar bordes

In [ ]:
image = np.invert(image)
image = segmentation.clear_border(image)
plt.imshow(image, cmap="gray")
plt.show()

#### Segmentación de células

In [ ]:
label,_ = ndi.label(image)
regions = measure.regionprops(label)
plt.imshow(label, cmap="gist_earth")
plt.show()

## Modulo 2 - Medición de forma celular
En este punto debe utilizar la segmentación resultante del punto anterior, y una segmentación de referencia que se entregó por u-cursos. Identifique cada una de las células en el epitelio (objetos conectados de un tamaño mínimo de 100 pixeles). Para cada uno de estos objetos obtenga las mediciones que se piden (posición, área, elongación).

En esta demostración, se entregarán primero los resultados y posteriormente el proceso paso a paso.

### Inicialización de Packages

In [ ]:
%matplotlib inline
from skimage import io, morphology, measure
from scipy import ndimage as ndi
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, widgets
from IPython import display
import mpld3
from mpld3 import plugins

#Ajustar el tamaño de las imágenes:
matplotlib.rcParams['figure.figsize'] = (14,12)

### Interactivo: Segmentación de celulas y filtrado de objetos pequeños
Si bien se solicita que se filtre sobre objetos mayores a 100px, se entrega una visualización sobre los resultados de filtrar sobre tamaños mayores.

In [ ]:
image = io.imread("./images/segmentacion_referencia.tif")
image = image[0]

def plot(min_size):
    fill = morphology.binary_erosion(image)
    fill = morphology.remove_small_objects(fill, connectivity=8, min_size=min_size)
    fill = morphology.binary_dilation(fill)
    
    label,_ = ndi.label(fill)
    regions = measure.regionprops(label)
    
    plt.imshow(label, cmap="inferno")
    
    print("Found:",len(regions),"regions!")
    plt.show()

slider = widgets.IntSlider(value=100, min=0, max=1000, description='Min Size:')
interact(plot, min_size=slider)

print("Interactive filtering")

### Interactivo: Identificación de características
En la siguiente visualización, se entrega una forma dinámica de identificar las distintas células según los parámetros solicitados.

In [ ]:
image = io.imread("./images/segmentacion_referencia.tif")
image = image[0]

fig, ax = plt.subplots()

fill = morphology.binary_erosion(image)
fill = morphology.remove_small_objects(fill,connectivity=8,min_size=100)
fill = morphology.binary_dilation(fill)
    
label,_ = ndi.label(fill)

regions = measure.regionprops(label)
regions.sort(key=lambda x: x.area, reverse=True)

newImg = np.zeros(image.shape, dtype=np.int32)
i=len(regions) + 20 # +20 para no llegar al negro

for region in regions:
    for coord in region.coords:
        newImg[coord[0],coord[1]] = i
    i -= 1
    
img = ax.imshow(newImg, cmap="gray")
N = len(regions)
scatter = ax.scatter(x=[i.centroid[1] for i in regions], 
           y=[i.centroid[0] for i in regions],
           alpha=0.1,
           s=[i.equivalent_diameter*50  for i in regions])

labels = ['<h1><p>&lambda;1: {0:.2f}</p><p>&lambda;2: {1:.2f}</p><p>Área: {2}</p> <p>Elongación: {3:.2f}</p></h1>'.format(i.major_axis_length, i.minor_axis_length,i.filled_area, (1 - (float(i.minor_axis_length)/float(i.major_axis_length)))) for i in regions if i.area>1]
print("(Los circulos azules son para visualizar levemente las etiquetas sobrepuestas)")
tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels=labels, css="h1 {color:black; text-shadow: 2px 0 0 #fff, -2px 0 0 #fff, 0 2px 0 #fff, 0 -2px 0 #fff, 1px 1px #fff, -1px -1px 0 #fff, 1px -1px 0 #fff, -1px 1px 0 #fff;}")
mpld3.plugins.connect(fig, tooltip)
mpld3.display()

### Proceso paso a paso

#### Visualización imágen

In [ ]:
image = io.imread("./images/segmentacion_referencia.tif")
image = image[0]
plt.imshow(image, cmap="gray")
plt.show()

#### Erosión
Se realiza erosión para marcar los bordes. Sin este proceso, no se pueden eliminar los objetos pequeños.

In [ ]:
fill = morphology.binary_erosion(image)
plt.imshow(fill, cmap="gray")
plt.show()

#### Eliminar objetos pequeños
En este caso, se filtran los objetos con tamaño menor a 100 en conectividad 8.

In [ ]:
fill = morphology.remove_small_objects(fill,connectivity=8,min_size=100)
plt.imshow(fill, cmap="gray")
plt.show()

#### Dilatación
Se procede a realizar una dilatación para contrarestar la erosión anterior.

In [ ]:
fill = morphology.binary_dilation(fill)
plt.imshow(fill, cmap="gray")
plt.show()

#### Etiquetar y obtener propiedades de las regiones

In [ ]:
label,_ = ndi.label(fill)
regions = measure.regionprops(label)
regions.sort(key=lambda x: x.area, reverse=True)

#### Descripción de las regiones
En este caso se filtran las regiones que tengan áreas menores a 1px.

In [ ]:
i = 1
for region in regions:
    if region.filled_area > 1:
        print()
        print("Región: ",i)
        print("Centroid:", region.centroid)
        print("Axis:",region.minor_axis_length,";", region.major_axis_length)
        print("Area:", region.filled_area)
        print("Elongación", 1-(region.minor_axis_length/region.major_axis_length))
        i+=1

#### Ordenamiento según tamaños

In [ ]:
newImg = np.zeros(image.shape, dtype=np.int32)
i=len(regions) + 20 # +20 para no llegar al negro

for region in regions:
    for coord in region.coords:
        newImg[coord[0],coord[1]] = i
    i -= 1

#### Visualizar resultado

In [ ]:
plt.imshow(newImg, cmap="inferno")
plt.show()